In [ ]:
#default_exp modelling.mybifacialvf
#default_cls_lvl 3

In [ ]:
#export
import bifacialvf

# bifacialvf modelling
> A wrapper around NREL's bifacialvf

Let's put some gps data

In [ ]:
#export
CHAMBERY = {'Name':'Chambery', 'latitude': 45.637001, 'longitude': 5.881, 'Elevation': 235.0, 'TZ':-1.0}

In [ ]:
#export
def get_tmy3(data):
    tmy3 = (data[['dni', 'dhi', 'zenith', 'azimuth', 'elevation']]
            .rename(columns={'dni':'DNI', 'dhi':'DHI'}))
    return tmy3

# Exports -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.core.ipynb.
Converted 01_utils.pandas_hacks.ipynb.
Converted 03_utils.matlab_io.ipynb.
Converted 04_utils.missing.ipynb.
Converted 05_utils.plot.ipynb.
Converted 06_modelling.mybifacialvf.ipynb.
Converted index.ipynb.
